In [1]:
import os
import glob
import numpy as np 
import psycopg2 as pg
import pandas as pd
from queries import *
%load_ext sql

In [2]:
conn = pg.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
    return all_files

# Process `song_data`

In [4]:
song_files = get_files('data/song_data/')

In [5]:
song_file_path = song_files[0]
print(song_file_path)

/Users/piyushbhandari/Desktop/data-modelling-postgres/data/song_data/A/A/A/TRAAAEF128F4273421.json


In [6]:
df = pd.read_json(song_file_path, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


In [7]:
num_songs, artist_id, artist_latitude, artist_longitude, artist_location, artist_name, song_id, title, duration, year = df.values[0]

## Insert into `artists` table

In [8]:
artist_data = [artist_id, artist_name, artist_location, artist_longitude, artist_latitude]
artist_data

['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan]

In [9]:
cur.execute(artist_table_insert, artist_data)

## Insert into `songs` table

In [10]:
song_data = [song_id, title, artist_id, year, duration] # song_id, title, artist_id, year, duration
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

In [11]:
cur.execute(song_table_insert, song_data)

# Process `log_data`

In [12]:
log_files = get_files('data/log_data/')

In [13]:
log_filepath = log_files[0]
print(log_filepath)

/Users/piyushbhandari/Desktop/data-modelling-postgres/data/log_data/2018/11/2018-11-11-events.json


In [14]:
log_df = pd.read_json(log_filepath, lines=True)
print(log_df.size)
log_df.head()

1710


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## Insert into `time` table

In [15]:
time_df = log_df[log_df['page'] == 'NextSong']
time_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [16]:
time = pd.to_datetime(time_df['ts'], unit='ms')
time.head()

0   2018-11-11 02:33:56.796
1   2018-11-11 02:36:10.796
2   2018-11-11 02:40:34.796
4   2018-11-11 04:36:13.796
5   2018-11-11 04:36:46.796
Name: ts, dtype: datetime64[ns]

In [17]:
# Extract timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data to a list containing these values in order
time_data = []
for line in time:
    time_data.append([line, line.hour, line.day, line.week, line.month, line.year, line.day_name()])

column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [18]:
time_df = pd.DataFrame.from_records(time_data, columns=column_labels)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-11 02:33:56.796,2,11,45,11,2018,Sunday
1,2018-11-11 02:36:10.796,2,11,45,11,2018,Sunday
2,2018-11-11 02:40:34.796,2,11,45,11,2018,Sunday
3,2018-11-11 04:36:13.796,4,11,45,11,2018,Sunday
4,2018-11-11 04:36:46.796,4,11,45,11,2018,Sunday


In [19]:
for item, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

## Insert into `users` table

In [20]:
user_df = log_df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.replace('', np.nan, inplace=True)
user_df.dropna(subset=['userId'], inplace=True)
user_df.head(30)

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
3,32,Lily,Burns,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free
6,92,Ryann,Smith,F,free
7,92,Ryann,Smith,F,free
8,92,Ryann,Smith,F,free
9,92,Ryann,Smith,F,free


In [21]:
for item, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

## Insert data into `songplays` table

In [29]:
for index, row in log_df.iterrows():

    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()

    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

In [23]:
# cur.close()
# conn.close()